In [1]:
import librosa
import pandas as pd
import numpy as np
from IPython.lib.display import Audio
from matplotlib import pyplot as plt
import multiprocessing
import scipy.signal
from scipy.fft import fft
import os

In [2]:
data_dir = "C:/Users/Jayde/Desktop/library/zebra_finch/data/"
anno = pd.read_csv(f'{data_dir}annotations.csv')

audio = []
srs= []

#adding the audio data to the dataframe
for _, row in anno.iterrows():
    recording, sr = librosa.load(f'{data_dir}vocalizations/{row.fn}')
    audio.append(recording)
    srs.append(sr)
    
anno['audio'] = audio



In [12]:
recs = []
srs = []
file_names= []
call_types = []
names = []


data_dir = "C:/Users/Jayde/Desktop/library/zebra_finch/data/AdultVocalizations/"
for file_name in os.listdir(data_dir):
    recording, sr = librosa.load(f'{data_dir}{file_name}', sr = None)
    #print(file)
    try:
        name, file= file_name.split("_")
        date, call_type, rendition= file.split("-")
        call_type = call_type[:2]
        #print(name, date, call_type)
        recs.append(recording)
        names.append(name)
        file_names.append(file_name)
        call_types.append(call_type)
        srs.append(sr)
    except:
        print(f"skipped {file}")
        continue

print(f"processed {len(recs)} out of {len(os.listdir(data_dir))}")

processed 2969 out of 2969


In [19]:
df = pd.DataFrame()
df["file_name"] = file_names
df["name"] = names
df["call_type"]= call_types
#df["sr"] = srs
df["rec"] = recs
durations = [x.shape[0] / 44100 for x in recs]
df["duration"] = durations


In [20]:
df.head()

,file_name,name,call_type,rec,duration
0,BlaBla0506_110302-AggC-04.wav,BlaBla0506,Ag,"[-1.5258789e-05, -0.0009918213, 3.0517578e-05,...",0.308118
1,BlaBla0506_110302-AggC-05.wav,BlaBla0506,Ag,"[0.0001373291, 0.00021362305, 0.00062561035, 0...",0.599184
2,BlaBla0506_110302-DC-01.wav,BlaBla0506,DC,"[0.00015258789, 0.0002746582, -0.0005340576, -...",0.238594
3,BlaBla0506_110302-DC-02.wav,BlaBla0506,DC,"[0.002105713, 0.0018157959, 0.0018157959, 0.00...",0.261111
4,BlaBla0506_110302-DC-05.wav,BlaBla0506,DC,"[-4.5776367e-05, -0.00032043457, -0.0006408691...",0.253220


In [21]:
df.describe()

,duration
count,2969.000000
mean,0.338989
std,0.719706
min,0.032494
25%,0.085442
50%,0.127551
75%,0.246100
max,11.705079


In [28]:
slice_lens = [16384, 32768, 65536]
for slice_len in slice_lens:
    dur = slice_len/44100
    count = (df["duration"]<= dur).sum()
    print(f" max dur:{dur} for slice len {slice_len}. {count} recordings ")

 max dur:0.37151927437641724 for slice len 16384. 2542 recordings 
 max dur:0.7430385487528345 for slice len 32768. 2699 recordings 
 max dur:1.486077097505669 for slice len 65536. 2807 recordings 


In [41]:
df= df[df["duration"] <= 16384/44100]

In [42]:
call_type_counts = df["call_type"].value_counts()
top_8 = call_type_counts.head(8).index.tolist()
top_8

['Te', 'DC', 'Ne', 'Th', 'Tu', 'Ag', 'Wh', 'LT']

In [43]:
df_top = df[df["call_type"].isin(top_8)]
df_top["call_type"].value_counts()

Te    613
DC    597
Ne    468
Th    301
Tu    240
Ag    164
Wh    131
LT     16
Name: call_type, dtype: int64

In [44]:
slice_lens = [16384, 32768, 65536]
for slice_len in slice_lens:
    dur = slice_len/44100
    count = (df_top["duration"]<= dur).sum()
    train_size = (count//64) *64
    print(f" max dur:{dur} for slice len {slice_len}. {count} recordings -> train size {train_size}")

 max dur:0.37151927437641724 for slice len 16384. 2530 recordings -> train size 2496
 max dur:0.7430385487528345 for slice len 32768. 2530 recordings -> train size 2496
 max dur:1.486077097505669 for slice len 65536. 2530 recordings -> train size 2496


In [45]:
df_top["call_type"].value_counts()

Te    613
DC    597
Ne    468
Th    301
Tu    240
Ag    164
Wh    131
LT     16
Name: call_type, dtype: int64